In [1]:
import numpy as np 
import matplotlib.pyplot as plt  
import ipywidgets as widgets
from ipywidgets import GridspecLayout


In [2]:
lambd_init = 1.4 # arrival rate in orders/week 
mu_init = 1.5 # production rate in orders/week

cv_arrival_init = 1 # coefficient of variation of arrival times
bw_init = 1.2 # bullwhip effect (multiplication of variation per stage)

style = {'description_width': 'initial'}



W_tier1 = widgets.FloatText(
    value=0,
    layout=widgets.Layout(display='None'))

W_oem = widgets.FloatText(
    value=0,
    layout=widgets.Layout(display='None'))

W_semicon = widgets.FloatText(
    value=0,
    layout=widgets.Layout(display='None'))


lambd_semicon = widgets.FloatSlider(
    value=lambd_init,
    min=0.01,
    max=mu_init,
    step=0.1,
    description='Order arrival rate in k wafers/week',
    disabled=True,
    layout=widgets.Layout(width='500px'),
    continuous_update=False, 
    style=style)

mu_semicon = widgets.FloatSlider(
    value=mu_init,
    min=0.01,
    max=10,
    step=0.1,
    description='Production rate in k wafers/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    continuous_update=False, 
    style=style)


cv_arrival_semicon = widgets.FloatSlider(
    value=cv_arrival_init*bw_init*bw_init,
    min=0.01,
    max=5,
    step=0.1,
    description='Coefficient of variation for order arrivals',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    continuous_update=False, 
    style=style)



lambd_oem = widgets.FloatSlider(
    value=lambd_init,
    min=0.01,
    max=mu_init,
    step=0.1,
    description='Order arrival rate in k cars/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    continuous_update=False, 
    style=style)

mu_oem = widgets.FloatSlider(
    value=mu_init,
    min=0.01,
    max=10,
    step=0.1,
    description='Production rate in k cars/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    continuous_update=False, 
    style=style)


cv_arrival_oem = widgets.FloatSlider(
    value=cv_arrival_init,
    min=0.01,
    max=5,
    step=0.1,
    description='Coefficient of variation for order arrivals',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    continuous_update=False, 
    style=style)


lambd_tier1 = widgets.FloatSlider(
    value=lambd_init,
    min=0.01,
    max=mu_init,
    step=0.1,
    description='Order arrival rate in k parts/week',
    disabled=True,
    layout=widgets.Layout(width='500px'),
    continuous_update=False, 
    style=style)

mu_tier1 = widgets.FloatSlider(
    value=mu_init,
    min=0.01,
    max=10,
    step=0.1,
    description='Production rate in k parts/week',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    continuous_update=False, 
    style=style)


cv_arrival_tier1 = widgets.FloatSlider(
    value=cv_arrival_init*bw_init,
    min=0.01,
    max=5,
    step=0.1,
    description='Coefficient of variation for order arrivals',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    continuous_update=False, 
    style=style)


In [3]:
def drawPlot(p,W,p_chart,W_chart,W_chart_novariation,cv_arrival):
    plt.plot(p_chart*100,W_chart) 
    plt.plot(p_chart*100,W_chart_novariation,color="g") 
    plt.plot(p*100, W, 'ro')
    plt.grid("on")
    plt.legend(["Waiting time for CV="+"{:.2f}".format(cv_arrival),"Waiting time for CV=1"])
    plt.title("Waiting time based on utilization") 
    plt.xlabel("Utilization (%)")
    plt.ylabel("Waiting time (weeks)")
    plt.ylim(bottom=0) 
    plt.axvline(p*100, color='r',lw=0.5) # vertical
    plt.axhline(W, color='r',lw=0.5) # horizontal

In [4]:
def updatePlot(lambd,mu,cv_arrival,W_oem,W_tier1,W_semicon,typ=""): 
    cv_production=1
    p=lambd/mu # utilization in % 
    
    if typ=="OEM": 
        W_prior=float(W_tier1)
    elif typ=="Tier1":
        W_prior=float(W_semicon)
    else:
        W_prior=0 
        
    W=W_prior+p/(1-p)*(np.power(cv_arrival,2)+np.power(cv_production,2))/2*1/mu
    p_chart=np.arange(0,1,0.01)
    W_chart=W_prior+p_chart/(1-p_chart)*(np.power(cv_arrival,2)+np.power(cv_production,2))/2*1/mu
    W_chart_novariation=W_prior+p_chart/(1-p_chart)*1/mu
    

    drawPlot(p,W,p_chart,W_chart,W_chart_novariation,cv_arrival) 
    plt.show()

    result=widgets.HTML(value="Total expected waiting time for "+typ+": "+ "{:.2f}".format(W)+" weeks")
    display(result) 


In [8]:
def updateMaxLambdaSemicon(*args): 
    lambd_semicon.max=min([mu_oem.value,mu_tier1.value,mu_semicon.value])-0.101
    mu_semicon.min = lambd_oem.value 
    
def updateMaxLambdaOEM(*args): 
    lambd_oem.max=min([mu_oem.value,mu_tier1.value,mu_semicon.value])-0.101    
def updateMaxLambdaTier1(*args): 
    lambd_tier1.max=min([mu_oem.value,mu_tier1.value,mu_semicon.value])-0.101 
    mu_tier1.min = lambd_oem.value 
        
def updateWaitTimes(*args): 
    lambd_semicon.value = lambd_oem.value
    lambd_tier1.value = lambd_oem.value 
    p_semicon=lambd_semicon.value/mu_semicon.value
    p_tier1=lambd_tier1.value/mu_tier1.value
    p_oem=lambd_oem.value/mu_oem.value
    cv_production=1 
    
    W_semicon.value=p_semicon/(1-p_semicon)*(np.power(cv_arrival_semicon.value,2)+np.power(cv_production,2))/2*1/mu_semicon.value
    W_tier1.value=W_semicon.value+p_tier1/(1-p_tier1)*(np.power(cv_arrival_tier1.value,2)+np.power(cv_production,2))/2*1/mu_tier1.value
    W_oem.value=W_tier1.value+p_oem/(1-p_oem)*(np.power(cv_arrival_oem.value,2)+np.power(cv_production,2))/2*1/mu_oem.value

    
mu_semicon.observe(updateMaxLambdaSemicon, 'value')
mu_tier1.observe(updateMaxLambdaTier1, 'value')
mu_oem.observe(updateMaxLambdaOEM, 'value')

lambd_oem.observe(updateMaxLambdaSemicon,"value") 
lambd_oem.observe(updateMaxLambdaTier1,"value") 
lambd_oem.observe(updateMaxLambdaOEM,"value")

mu_semicon.observe(updateWaitTimes, 'value')
mu_tier1.observe(updateWaitTimes, 'value')
mu_oem.observe(updateWaitTimes, 'value')

cv_arrival_tier1.observe(updateWaitTimes,"value") 
cv_arrival_oem.observe(updateWaitTimes,"value") 
cv_arrival_semicon.observe(updateWaitTimes,"value") 

lambd_semicon.observe(updateWaitTimes,"value")
lambd_tier1.observe(updateWaitTimes,"value") 
lambd_oem.observe(updateWaitTimes,"value") 
   


In [9]:
updateWaitTimes() 

grid = GridspecLayout(3, 3,height="150px")
grid[2,0]= widgets.HTML(value="<b>OEM</b>")
grid[2,1]= widgets.HTML(value="<b>Tier1-Supplier</b>")
grid[2,2]= widgets.HTML(value="<b>Semicon-Supplier</b>")




display(grid) 
grid2 = GridspecLayout(1, 3)
grid2[0, 0] =  widgets.interactive(updatePlot, lambd=lambd_oem,mu=mu_oem, cv_arrival=cv_arrival_oem,W_oem=W_oem,W_tier1=W_tier1,W_semicon=W_semicon,typ=widgets.fixed("OEM"))
grid2[0, 1] =  widgets.interactive(updatePlot, lambd=lambd_tier1,mu=mu_tier1, cv_arrival=cv_arrival_tier1,W_oem=W_oem,W_tier1=W_tier1,W_semicon=W_semicon,typ=widgets.fixed("Tier1"))
grid2[0, 2] =  widgets.interactive(updatePlot, lambd=lambd_semicon,mu=mu_semicon, cv_arrival=cv_arrival_semicon,W_oem=W_oem,W_tier1=W_tier1,W_semicon=W_semicon,typ=widgets.fixed("Semicon"))
display(grid2) 



GridspecLayout(children=(HTML(value='<b>Bullwhip-effect</b>', layout=Layout(grid_area='widget001')), FloatSlid…

GridspecLayout(children=(interactive(children=(FloatSlider(value=1.4, continuous_update=False, description='Or…